# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 8:50AM,246112,12,HH-36838,Hush Hush,Released
1,Sep 12 2022 8:50AM,246112,12,HH-36839,Hush Hush,Released
2,Sep 12 2022 8:50AM,246112,12,HH-36840,Hush Hush,Released
3,Sep 12 2022 8:50AM,246112,12,HH-36841,Hush Hush,Released
4,Sep 12 2022 8:50AM,246112,12,HH-36842,Hush Hush,Released
5,Sep 12 2022 8:50AM,246112,12,HH-36844,Hush Hush,Released
6,Sep 12 2022 8:50AM,246112,12,HH-36845,Hush Hush,Released
7,Sep 12 2022 8:50AM,246112,12,HH-36846,Hush Hush,Released
8,Sep 12 2022 8:50AM,246112,12,HH-36848,Hush Hush,Released
9,Sep 12 2022 8:50AM,246112,12,HH-36849,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,246109,Released,17
9,246110,Released,1
10,246111,Released,36
11,246112,Released,35
12,246113,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246109,NaN,17.0
246110,NaN,1.0
246111,NaN,36.0
246112,NaN,35.0
246113,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246102,3.0,22.0
246103,1.0,6.0
246104,1.0,0.0
246106,0.0,1.0
246107,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246102,3,22
246103,1,6
246104,1,0
246106,0,1
246107,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246102,3,22
1,246103,1,6
2,246104,1,0
3,246106,0,1
4,246107,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246102,3,22
1,246103,1,6
2,246104,1,
3,246106,,1
4,246107,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 8:50AM,246112,12,Hush Hush
35,Sep 12 2022 8:48AM,246113,12,Hush Hush
37,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation
73,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation
90,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc."
91,Sep 12 2022 8:25AM,246107,20,"ACI Healthcare USA, Inc."
92,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc."
93,Sep 12 2022 8:21AM,246108,10,Beach Products Inc
106,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd"
113,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 12 2022 8:50AM,246112,12,Hush Hush,,35
1,Sep 12 2022 8:48AM,246113,12,Hush Hush,,2
2,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation,,36
3,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation,,17
4,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc.",,1
5,Sep 12 2022 8:25AM,246107,20,"ACI Healthcare USA, Inc.",,1
6,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",,1
7,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,,13
8,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",1,6
9,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:50AM,246112,12,Hush Hush,35,
1,Sep 12 2022 8:48AM,246113,12,Hush Hush,2,
2,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation,36,
3,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation,17,
4,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc.",1,
5,Sep 12 2022 8:25AM,246107,20,"ACI Healthcare USA, Inc.",1,
6,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",1,
7,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,13,
8,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",6,1
9,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:50AM,246112,12,Hush Hush,35,
1,Sep 12 2022 8:48AM,246113,12,Hush Hush,2,
2,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation,36,
3,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation,17,
4,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:50AM,246112,12,Hush Hush,35.0,NaN
1,Sep 12 2022 8:48AM,246113,12,Hush Hush,2.0,NaN
2,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation,36.0,NaN
3,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation,17.0,NaN
4,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:50AM,246112,12,Hush Hush,35.0,0.0
1,Sep 12 2022 8:48AM,246113,12,Hush Hush,2.0,0.0
2,Sep 12 2022 8:41AM,246111,10,ISDIN Corporation,36.0,0.0
3,Sep 12 2022 8:26AM,246109,10,ISDIN Corporation,17.0,0.0
4,Sep 12 2022 8:26AM,246106,20,"ACI Healthcare USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,984438,67.0,0.0
12,492225,37.0,0.0
19,492205,28.0,4.0
20,492213,2.0,0.0
22,246104,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,984438,67.0,0.0
1,12,492225,37.0,0.0
2,19,492205,28.0,4.0
3,20,492213,2.0,0.0
4,22,246104,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,67.0,0.0
1,12,37.0,0.0
2,19,28.0,4.0
3,20,2.0,0.0
4,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,12,Released,37.0
2,19,Released,28.0
3,20,Released,2.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,0.0,0.0,4.0,0.0,1.0
Released,67.0,37.0,28.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,0.0,0.0,4.0,0.0,1.0
1,Released,67.0,37.0,28.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,0.0,0.0,4.0,0.0,1.0
1,Released,67.0,37.0,28.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()